In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
from multiprocessing import Pool, Lock, Value
from time import sleep
import time
import pandas as pd
import requests
import re
import html

In [4]:
authors_id = [29369, 29442, 102994, 26149, 35952]
books_urls = []
books_ids = []

with webdriver.Firefox() as driver:
    for page in authors_id:
        driver.get("https://www.bookvoed.ru/author/books?id={}".format(page))
        sleep(1)
        last_height = 0
        while True:
            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                sleep(2)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height
            try:
                button = driver.find_element_by_css_selector('a.ty')
                sleep(1)
                button.click()
                sleep(1)
            except:
                print("end of page {}".format(page))
                break
        content = driver.find_elements_by_css_selector('div.gf')
        #content = content.get_attribute("data-book")
        for ids in content:
            books_urls.append('https://www.bookvoed.ru/book?id={}'.format(ids.get_attribute("data-book")))
            books_ids.append(ids.get_attribute("data-book"))

end of page 29369
end of page 29442
end of page 102994
end of page 26149
end of page 35952


In [5]:
print(len(books_urls))
for i in books_urls[:5]:
    print(i)

1418
https://www.bookvoed.ru/book?id=9340995
https://www.bookvoed.ru/book?id=9340993
https://www.bookvoed.ru/book?id=9689115
https://www.bookvoed.ru/book?id=8867666
https://www.bookvoed.ru/book?id=8550136


In [6]:
agelimit = {'sv nM': '18+', 'rv nM': '16+', 'qv nM': '12+', 'pv nM': '6+', 'ov nM': '0+'}

In [10]:
def extract_book_info(number):
    book_html = requests.get(books_urls[number]).text
    soup = BeautifulSoup(book_html, 'html.parser')
    desc = soup.find('div', class_='lw')
    if desc is not None:
        for nn in desc.find_all('div'):
            nn.replace_with(' ')
        desc = re.sub('\s+', ' ', desc.text)   
    table = soup.find('table', class_='tw')
    if table is not None:
        rows = table.find_all('tr')
        if rows is not None:
            data = [list(map(lambda x: x.text, row.find_all('td'))) for row in rows]
            book_info1 = dict(data)
    book_info = {}
    if len(book_info1) != 0:
        for old_key in book_info1:
            new_key = old_key[:-1]
            book_info[new_key] = book_info1[old_key]
        for lim in agelimit:
            if soup.find('div', class_=lim) is not None:
                agel = agelimit[lim]
    book_info['Возраст'] = agel
    book_info['ID']  = int(books_ids[number])
    f = soup.find('h1', itemprop='name')
    if f is not None:
        book_info['Название'] = f.text
    f = soup.find('img', class_='tf')
    if f is not None:
        book_info['Обложка'] = f.attrs['src']
    f = soup.find('div', class_='af')
    if f is not None:
        s = re.findall(r'\d{2}\.\d{2}%', f.attrs['style'])
        if len(s) != 0:
            book_info['Рейтинг'] = s[0]
    f = soup.find('a', class_='Ke Me ')
    if f is not None:
        s = re.findall(r'\d{1,7}', f.text)
        if len(s) != 0:
            book_info['Понравилось'] = int(s[0])
    f = soup.find('a', class_='Ke Le ff')
    if f is not None:
        s = re.findall(r'\d{1,7}', f.text)
        if len(s) != 0:
            book_info['В закладки'] = int(s[0])
    f = soup.find('a', class_='Ke Oe ')
    if f is not None:
        s = re.findall(r'\d{1,7}', f.text)
        if len(s) != 0:
            book_info['Не понравилось'] = int(s[0])
    f = soup.find('div', class_='Hu Wu')
    if f is not None:
        s = re.findall(r'\d{3,5} ....', f.text)
        if len(s) != 0:
            book_info['Цена'] = s[0]
    book_info['Описание'] = desc
    return book_info

In [12]:
t = time.time()

mutex = Lock()
n_processed = Value('i', 0)


def func_wrapper(uid):
    res = extract_book_info(uid)
    with mutex:
        global n_processed
        n_processed.value += 1
        if n_processed.value % 10 == 0:
            print(f"\r{n_processed.value} objects are processed...", end='', flush=True)
    return res

with Pool(processes=10) as pool:
    res = pool.map(func_wrapper, range(len(books_urls)))

df = pd.DataFrame(res)
df.sort_values(by=['ID'], inplace=True)
with open('data/hw_3.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv, index=False)    

print('\n')
print(time.time()-t)

1410 objects are processed...

293.2938320636749


In [14]:
df.head(50)

,ID,ISBN,Автор,Авторы,В базе,В закладки,Возраст,Год,Издательство,Код,...,Понравилось,Производитель,Размеры,Рейтинг,Серия,Страниц,Тематика,Тираж,Формат,Цена
1417,330661,978-5-699-20169-3,Емец Дмитрий Александрович,NaN,Э.Мефодий Буслаев-7.Лед и пламя Тартара,26,12+,2013,Эксмо,323903,...,359.0,NaN,"13,00 см x 20,00 см x 2,60 см",98.09%,Мефодий Буслаев,416,Для среднего школьного возраста,90100,205.00mm x 135.00mm x 25.00mm,NaN
1416,331020,978-5-699-18592-4,Емец Дмитрий Александрович,NaN,Э.Мефодий Буслаев-6.Тайная магия Депресняка,22,12+,2010,Эксмо,296750,...,316.0,NaN,"13,00 см x 20,00 см x 2,70 см",98.44%,Мефодий Буслаев,416,Для среднего школьного возраста,85000,84х108/32,NaN
1415,331077,978-5-699-16154-6,Емец Дмитрий Александрович,NaN,Э.Мефодий Буслаев-5.Месть валькирий,23,12+,2013,Эксмо,276705,...,394.0,NaN,"13,00 см x 20,00 см x 2,70 см",97.52%,Мефодий Буслаев,416,Для среднего школьного возраста,80000,205.00mm x 135.00mm x 30.00mm,NaN
1414,331239,978-5-699-14552-2,Емец Дмитрий Александрович,NaN,Э.Мефодий Буслаев-4.Билет на Лысую Гору,21,12+,2013,Эксмо,269060,...,329.0,NaN,"13,00 см x 20,00 см x 2,90 см",97.34%,Мефодий Буслаев,416,Для среднего школьного возраста,85000,205.00mm x 135.00mm x 30.00mm,NaN
1413,331289,978-5-699-10198-6,Емец Дмитрий Александрович,NaN,Э.Мефодий Буслаев.Свиток желаний,8,12+,2013,Эксмо,198701,...,160.0,NaN,"13,00 см x 20,00 см x 2,50 см",98.77%,Мефодий Буслаев,416,Для среднего школьного возраста,NaN,205.00mm x 135.00mm x 30.00mm,NaN
1412,334427,978-5-699-23298-7,Емец Дмитрий Александрович,NaN,Э.Мефодий Буслаев-8.Первый эйдос,22,12+,2010,Эксмо,351244,...,358.0,NaN,"13,00 см x 20,00 см x 2,50 см",98.08%,Мефодий Буслаев,416,Для среднего школьного возраста,85100,205.00mm x 130.00mm x 25.00mm,NaN
1411,334863,978-5-699-24621-2,Емец Дмитрий Александрович,NaN,Э.Мефодий Буслаев-9.Светлые крылья для темн,16,12+,2014,Эксмо,361354,...,171.0,NaN,"13,00 см x 20,00 см x 3,00 см",97.16%,Мефодий Буслаев,416,Для среднего школьного возраста,90100,205.00mm x 135.00mm x 30.00mm,NaN
1410,337086,978-5-699-24897-1,Емец Дмитрий Александрович,NaN,Э.Меф.Буслаев.Улетные фраз.и афор.Таня Гроттер...,3,12+,2007,Эксмо,372484,...,28.0,NaN,"13,00 см x 20,00 см x 2,90 см",90.32%,NaN,384,Для среднего школьного возраста,15000,84х108/32,NaN
1409,338563,978-5-699-26839-9,Емец Дмитрий Александрович,NaN,Э.ТГ.Таня Гроттер13.И болтливый сфинкс,3,12+,2008,Эксмо,382689,...,42.0,NaN,"13,00 см x 20,00 см x 2,80 см",NaN,Таня Гроттер,352,Для среднего школьного возраста,90100,84х108/32,NaN
736,338670,978-5-699-27395-9,Роллинс Джеймс,NaN,Э.КЗКБ.Черный орден,17,16+,2017,Эксмо,388098,...,73.0,NaN,"21,50 см x 14,50 см x 3,70 см",98.65%,"Книга-загадка, книга-бестселлер",480,Зарубежная,5000,219.00mm x 155.00mm x 36.00mm,NaN
